In [12]:
import yfinance as yf
import pandas as pd
import numpy as np

In [13]:
import pandas as pd
import json
from datetime import datetime, timedelta

# Load the JSON data from the file
# file_path = 'snow_volume_5_min'
file_path = 'TRV_Volume_Footprint'


with open(file_path, 'r') as file:
    file_content = file.read()

# Strip everything after the ~m~98~m~ marker
marker = '~m~98~m~'
# marker = '~m~152~m~'
file_content = file_content.split(marker)[0]

# file_content

main_data = json.loads(file_content)

data_section = main_data['p'][1]['st1']['ns']['d']
nested_data = json.loads(data_section)
footprint_levels = nested_data['graphicsCmds']['create']['footprintLevels']
df = pd.DataFrame(footprint_levels[0]['data'])

footprints = nested_data['graphicsCmds']['create']['footprints']

df1 = pd.DataFrame(footprints[0]['data'])


In [14]:
df1

,id,index,poc,val,vah,levels
0,2,0,213.42,213.40,213.44,"[{'buyVolume': 14.0, 'sellVolume': 0.0, 'imbal..."
1,7,1,213.42,213.40,213.44,"[{'buyVolume': 4.0, 'sellVolume': 0.0, 'imbala..."
2,8,2,213.30,213.28,213.32,"[{'buyVolume': 1590.0, 'sellVolume': 20.0, 'im..."
3,9,3,213.26,213.24,213.28,"[{'buyVolume': 12.0, 'sellVolume': 10.0, 'imba..."
4,10,4,212.98,212.96,213.00,"[{'buyVolume': 3.0, 'sellVolume': 5.0, 'imbala..."
...,...,...,...,...,...,...
2348,7787,2348,208.22,208.16,208.24,"[{'buyVolume': 164.0, 'sellVolume': 0.0, 'imba..."
2349,7788,2349,208.02,208.00,208.08,"[{'buyVolume': 173.0, 'sellVolume': 0.0, 'imba..."
2350,7792,2350,208.38,208.16,208.48,"[{'buyVolume': 198.0, 'sellVolume': 0.0, 'imba..."
2351,7797,2351,208.46,208.40,208.48,"[{'buyVolume': 42.0, 'sellVolume': 452.0, 'imb..."


In [15]:
df.head()

,id,startIndex,type,price,extendTo
0,3,0,buy,213.36,1.0
1,4,0,buy,213.40,1.0
2,5,0,buy,213.48,7.0
3,6,0,buy,213.52,7.0
4,17,7,sell,213.36,8.0


In [16]:
df1.head()

,id,index,poc,val,vah,levels
0,2,0,213.42,213.40,213.44,"[{'buyVolume': 14.0, 'sellVolume': 0.0, 'imbal..."
1,7,1,213.42,213.40,213.44,"[{'buyVolume': 4.0, 'sellVolume': 0.0, 'imbala..."
2,8,2,213.30,213.28,213.32,"[{'buyVolume': 1590.0, 'sellVolume': 20.0, 'im..."
3,9,3,213.26,213.24,213.28,"[{'buyVolume': 12.0, 'sellVolume': 10.0, 'imba..."
4,10,4,212.98,212.96,213.00,"[{'buyVolume': 3.0, 'sellVolume': 5.0, 'imbala..."


In [17]:
# Define the trading holidays
trading_holidays = [
    datetime(2024, 1, 1),   # Monday, January 1
    datetime(2024, 1, 15),  # Monday, January 15
    datetime(2024, 2, 19),  # Monday, February 19
    datetime(2024, 3, 29),  # Friday, March 29
    datetime(2024, 5, 27),  # Monday, May 27
    datetime(2024, 6, 19),  # Wednesday, June 19
    datetime(2024, 7, 4)    # Thursday, July 4
]

# Initialize the base timestamp for index 2245
base_timestamp = datetime.strptime('2024-07-23 15:55:00', '%Y-%m-%d %H:%M:%S')
current_timestamp = base_timestamp

# Initialize a dictionary to store timestamps for each index
# index_to_timestamp = {2245: current_timestamp}
index_to_timestamp = {max(df1['index']): current_timestamp}

# Market hours
market_open = timedelta(hours=9, minutes=30)
market_close = timedelta(hours=15, minutes=55)
day_increment = timedelta(days=1)
weekend_days = [5, 6]  # Saturday and Sunday

# Calculate the timestamps backward in 5-minute intervals, excluding weekends and outside market hours
# for index in range(2244, -1, -1):
for index in range(max(df1['index'])-1, -1, -1):
    # Subtract 5 minutes
    current_timestamp -= timedelta(minutes=5)
    
    # Check if current timestamp is before market open
    while (current_timestamp.time() < (datetime.min + market_open).time() or
           current_timestamp.time() > (datetime.min + market_close).time() or
           current_timestamp.weekday() in weekend_days or
           current_timestamp.date() in [holiday.date() for holiday in trading_holidays]):
        # Move to previous trading day if before market open
        if current_timestamp.time() < (datetime.min + market_open).time():
            current_timestamp = datetime.combine(current_timestamp.date() - day_increment, (datetime.min + market_close).time())
        else:
            # Otherwise, just subtract 5 minutes
            current_timestamp -= timedelta(minutes=5)
        
        # Skip weekends and trading holidays
        while current_timestamp.weekday() in weekend_days or current_timestamp.date() in [holiday.date() for holiday in trading_holidays]:
            current_timestamp -= day_increment
            current_timestamp = datetime.combine(current_timestamp.date(), (datetime.min + market_close).time())
    
    # Assign the calculated timestamp to the index
    index_to_timestamp[index] = current_timestamp

# Create a list to hold the time series data
time_series_data = []

# Iterate over df1 and extract levels data
for i, row in df1.iterrows():
    timestamp = index_to_timestamp.get(row['index'])
    
    if timestamp:
        levels = row['levels']
        for level in levels:
            time_series_data.append({
                'timestamp': timestamp,
                'price': level['price'],
                'buyVolume': level['buyVolume'],
                'sellVolume': level['sellVolume'],
                'imbalance': level['imbalance'],
                'index': row['index']
            })

# Create the dataframe from the time series data
series_df = pd.DataFrame(time_series_data)

series_df.tail(20)

,timestamp,price,buyVolume,sellVolume,imbalance,index
16979,2024-07-23 15:45:00,208.36,82.0,650.0,sell,2350
16980,2024-07-23 15:45:00,208.40,140.0,60.0,,2350
16981,2024-07-23 15:45:00,208.44,75.0,0.0,,2350
16982,2024-07-23 15:45:00,208.32,434.0,200.0,buy,2350
16983,2024-07-23 15:50:00,208.36,42.0,452.0,buy,2351
16984,2024-07-23 15:50:00,208.40,219.0,1158.0,,2351
16985,2024-07-23 15:50:00,208.44,885.0,1347.0,sell,2351
16986,2024-07-23 15:50:00,208.48,275.0,899.0,sell,2351
16987,2024-07-23 15:50:00,208.52,32.0,100.0,,2351
16988,2024-07-23 15:50:00,208.32,0.0,5.0,,2351


In [18]:
series_df.to_csv('trv_5_min_series_data.csv')

In [56]:
# Filter the DataFrame
filtered_df = series_df[series_df['timestamp'] >= '2024-07-23 09:30:00']

In [57]:
filtered_df.reset_index(inplace=True,drop=True)

In [58]:
filtered_df

,timestamp,price,buyVolume,sellVolume,imbalance,index
0,2024-07-23 09:30:00,205.88,50.0,0.0,buy,2275
1,2024-07-23 09:30:00,206.36,6.0,9.0,buy,2275
2,2024-07-23 09:30:00,205.68,0.0,1.0,,2275
3,2024-07-23 09:30:00,205.64,0.0,127.0,sell,2275
4,2024-07-23 09:30:00,205.56,0.0,71.0,sell,2275
...,...,...,...,...,...,...
445,2024-07-23 15:55:00,208.60,2257.0,2004.0,sell,2352
446,2024-07-23 15:55:00,208.64,307.0,286.0,sell,2352
447,2024-07-23 15:55:00,208.68,46.0,274.0,,2352
448,2024-07-23 15:55:00,208.72,151.0,1017.0,sell,2352


In [59]:
from sklearn.ensemble import IsolationForest

# Replace missing values in 'imbalance' with a specific value (e.g., 'missing_value')
filtered_df['imbalance'] = filtered_df['imbalance'].fillna('missing_value')

# Encode categorical 'imbalance' feature
filtered_df['imbalance'] = filtered_df['imbalance'].apply(lambda x: 1 if x == 'buy' else -1 if x == 'sell' else 0)

# Convert the timestamp to numerical format (e.g., seconds since epoch)
filtered_df['timestamp'] = filtered_df['timestamp'].astype(np.int64) // 10**9

# Select relevant features for anomaly detection
features = ['timestamp', 'price', 'buyVolume', 'sellVolume', 'imbalance']

# # Initialize and fit the Isolation Forest model
# model = IsolationForest(contamination='auto', random_state=42)
model = IsolationForest(contamination=0.03, random_state=42)
model.fit(filtered_df[features])

# Predict anomalies
filtered_df['anomaly'] = model.predict(filtered_df[features])

/var/folders/gp/x0f5sgq13db073j79l3_2tb40000gn/T/ipykernel_3751/2088222860.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['imbalance'] = filtered_df['imbalance'].fillna('missing_value')
/var/folders/gp/x0f5sgq13db073j79l3_2tb40000gn/T/ipykernel_3751/2088222860.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['imbalance'] = filtered_df['imbalance'].apply(lambda x: 1 if x == 'buy' else -1 if x == 'sell' else 0)
/var/folders/gp/x0f5sgq13db073j79l3_2tb40000gn/T/ipykernel_3751/208

In [60]:
filtered_df['anomaly'].value_counts()

 1    436
-1     14
Name: anomaly, dtype: int64

In [61]:
# Convert the numerical timestamp back to normal datetime format
filtered_df['timestamp'] = pd.to_datetime(filtered_df['timestamp'], unit='s')

filtered_anomaly = filtered_df[filtered_df['anomaly'] == -1]

/var/folders/gp/x0f5sgq13db073j79l3_2tb40000gn/T/ipykernel_3751/1014059852.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['timestamp'] = pd.to_datetime(filtered_df['timestamp'], unit='s')


In [62]:
filtered_anomaly['timestamp'].value_counts()

2024-07-23 15:55:00    7
2024-07-23 15:50:00    3
2024-07-23 10:00:00    1
2024-07-23 10:55:00    1
2024-07-23 15:30:00    1
2024-07-23 15:45:00    1
Name: timestamp, dtype: int64

In [63]:
filtered_anomaly

,timestamp,price,buyVolume,sellVolume,imbalance,index,anomaly
45,2024-07-23 10:00:00,207.16,3316.0,0.0,1,2281,-1
128,2024-07-23 10:55:00,206.36,291.0,522.0,1,2292,-1
411,2024-07-23 15:30:00,207.96,268.0,1018.0,0,2347,-1
430,2024-07-23 15:45:00,208.36,82.0,650.0,-1,2350,-1
435,2024-07-23 15:50:00,208.40,219.0,1158.0,0,2351,-1
436,2024-07-23 15:50:00,208.44,885.0,1347.0,-1,2351,-1
437,2024-07-23 15:50:00,208.48,275.0,899.0,-1,2351,-1
441,2024-07-23 15:55:00,208.44,974.0,46.0,1,2352,-1
442,2024-07-23 15:55:00,208.48,2318.0,2814.0,1,2352,-1
443,2024-07-23 15:55:00,208.52,2598.0,1851.0,0,2352,-1


In [70]:
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, RepeatVector, TimeDistributed


# Filter the DataFrame
filtered_df = series_df[(series_df['timestamp'] >= '2024-07-23 09:30:00') & (series_df['timestamp'] < '2024-07-23 15:50:00')]
filtered_df.reset_index(inplace=True,drop=True)

# Replace missing values in 'imbalance' with a specific value (e.g., 'missing_value')
filtered_df['imbalance'] = filtered_df['imbalance'].fillna('missing_value')

# Encode categorical 'imbalance' feature
filtered_df['imbalance'] = filtered_df['imbalance'].apply(lambda x: 1 if x == 'buy' else -1 if x == 'sell' else 0)

# Select relevant features for anomaly detection
features = ['price', 'buyVolume', 'sellVolume', 'imbalance']

# Normalize the data
scaler = MinMaxScaler()
filtered_df[features] = scaler.fit_transform(filtered_df[features])

/var/folders/gp/x0f5sgq13db073j79l3_2tb40000gn/T/ipykernel_3751/451711203.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['imbalance'] = filtered_df['imbalance'].fillna('missing_value')
/var/folders/gp/x0f5sgq13db073j79l3_2tb40000gn/T/ipykernel_3751/451711203.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['imbalance'] = filtered_df['imbalance'].apply(lambda x: 1 if x == 'buy' else -1 if x == 'sell' else 0)
/var/folders/gp/x0f5sgq13db073j79l3_2tb40000gn/T/ipykernel_3751/451

In [71]:
# Prepare the data for LSTM (we'll use sequences)
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:(i + seq_length)]
        y = data[i + seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

SEQ_LENGTH = 10  # Length of the sequences for LSTM
X, y = create_sequences(filtered_df[features].values, SEQ_LENGTH)

# Build LSTM Autoencoder Model
model = Sequential([
    LSTM(128, activation='relu', input_shape=(SEQ_LENGTH, len(features)), return_sequences=True),
    Dropout(0.2),
    LSTM(64, activation='relu', return_sequences=False),
    RepeatVector(SEQ_LENGTH),
    LSTM(64, activation='relu', return_sequences=True),
    Dropout(0.2),
    LSTM(128, activation='relu', return_sequences=True),
    TimeDistributed(Dense(len(features)))
])

model.compile(optimizer='adam', loss='mse')
model.summary()

# Train the model
history = model.fit(X, X, epochs=50, batch_size=32, validation_split=0.1, shuffle=False)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 10, 128)           68096     
                                                                 
 dropout_2 (Dropout)         (None, 10, 128)           0         
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 repeat_vector_1 (RepeatVect  (None, 10, 64)           0         
 or)                                                             
                                                                 
 lstm_6 (LSTM)               (None, 10, 64)            33024     
                                                                 
 dropout_3 (Dropout)         (None, 10, 64)            0         
                                                      

In [73]:
# Predict the reconstruction
X_pred = model.predict(X)

# Calculate the reconstruction error
reconstruction_error = np.mean(np.abs(X_pred - X), axis=(1, 2))

# Threshold for anomaly detection
threshold = np.mean(reconstruction_error) + 3 * np.std(reconstruction_error)

# Detect anomalies
anomalies = reconstruction_error > threshold

# Append anomaly scores to the original DataFrame
anomaly_scores = np.zeros(len(filtered_df))
anomaly_scores[SEQ_LENGTH:] = anomalies

filtered_df['anomaly'] = anomaly_scores

14/14 [==============================] - 0s 7ms/step


/var/folders/gp/x0f5sgq13db073j79l3_2tb40000gn/T/ipykernel_3751/1529233682.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['anomaly'] = anomaly_scores


In [75]:

# Inverse transform the normalized features back to their original scale
filtered_df[features] = scaler.inverse_transform(filtered_df[features])


/var/folders/gp/x0f5sgq13db073j79l3_2tb40000gn/T/ipykernel_3751/984967263.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[features] = scaler.inverse_transform(filtered_df[features])


In [76]:
filtered_df[filtered_df['anomaly'] == 1]

,timestamp,price,buyVolume,sellVolume,imbalance,index,anomaly
138,2024-07-23 11:00:00,206.48,8.0,0.0,1.0,2293,1.0
